***vis_completables_times* function**:
* Input: players_info dictionary
* Displays bar chart showing for each completable the maximum, minimum and mean time of completion by players

In [1]:
def vis_completables_times(players_info):
    # find full list of completables
    completables=set()
    for player in players_info.keys():
        if players_info[player]["completables_times"] != []:
            completables = completables.union(players_info[player]["completables_times"].keys())
    completables = list(completables)

    # max, min time per completable
    max_times = []
    min_times = []
    for c in completables:
        maximum = 0
        minimum = float('Inf')
        for player in players_info.keys():
            if c in players_info[player]["completables_times"].keys():
                times = players_info[player]["completables_times"][c]
                if len(times)==2:
                    end_time = datetime.strptime(times[1], '%Y-%m-%dT%H:%M:%SZ')
                    start_time = datetime.strptime(times[0], '%Y-%m-%dT%H:%M:%SZ')
                    raw_total_time = end_time-start_time
                    total_time = raw_total_time.days*24*60+raw_total_time.seconds/60.0 # minutes
                    # update values
                    if total_time > maximum:
                        maximum = total_time
                    if total_time < minimum:
                        minimum = total_time
        max_times.append(maximum)
        min_times.append(minimum)
        
        
    # Bar chart showing for each completable the maximum, minimum and mean time of completion by players.

    n_groups = len(completables)

    fig, ax = plt.subplots()
    index = np.arange(n_groups)
    bar_width = 0.35
    opacity = 0.8

    times=[max_times,min_times]
    labels=['max', 'min']

    for i in range(len(labels)):
        plt.bar(index+i*bar_width, times[i], bar_width, 
                label=labels[i], 
                tick_label=completables)

    plt.legend()
    plt.xlabel("Completable")
    plt.ylabel("Completion time")
    plt.title("Max and min completion times per completable")
    plt.show()